In [ ]:
import pandas as pd

In [ ]:
import sys
print(sys.version)


3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [ ]:
!pip install -q librosa==0.10.1 soundfile audioread numpy scipy scikit-learn joblib pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.7/253.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
import os

os.makedirs("data/audio/original", exist_ok=True)
os.makedirs("data/audio/prompt_injection", exist_ok=True)
os.makedirs("models", exist_ok=True)


In [ ]:
import librosa

audio_path = "/content/original.m4a"  # change name if needed

y, sr = librosa.load(audio_path, sr=16000, mono=True)

print("Audio loaded")
print("Sample rate:", sr)
print("Duration:", len(y)/sr)


/tmp/ipython-input-4217415713.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio loaded
Sample rate: 16000
Duration: 48.0426875


In [ ]:
import numpy as np

def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=16000, mono=True)
    y = librosa.util.normalize(y)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    delta = librosa.feature.delta(mfcc)
    zcr = librosa.feature.zero_crossing_rate(y)
    spectral_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)

    features = np.hstack([
        mfcc.mean(axis=1),
        delta.mean(axis=1),
        zcr.mean(),
        spectral_bw.mean()
    ])

    return features


In [ ]:
X, y = [], []

for label, folder in enumerate(["original", "prompt_injection"]):
    path = f"data/audio/{folder}"
    for file in os.listdir(path):
        if file.endswith((".mp3", ".m4a", ".wav")):
            feat = extract_features(os.path.join(path, file))
            X.append(feat)
            y.append(label)

X = np.array(X)
y = np.array(y)

print("Samples:", X.shape)
print("Labels:", set(y))


/tmp/ipython-input-2669740500.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-2669740500.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-2669740500.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path, sr=16000, mono=True)
/usr/loca

Samples: (12, 28)
Labels: {np.int64(0), np.int64(1)}


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

model = RandomForestClassifier(
    n_estimators=200,
    max_depth=12,
    random_state=42
)

model.fit(X_train, y_train)

joblib.dump(model, "models/audio_classifier.pkl")
joblib.dump(scaler, "models/scaler.pkl")

print("Model trained & saved")


Model trained & saved


In [ ]:
def replay_score(audio_path):
    y, sr = librosa.load(audio_path, sr=16000, mono=True)
    spec = np.abs(librosa.stft(y))

    high = spec[int(0.75 * spec.shape[0]):].mean()
    low = spec[:int(0.25 * spec.shape[0])].mean()

    return float(high / (low + 1e-6))


In [ ]:
def hidden_command_score(audio_path):
    y, sr = librosa.load(audio_path, sr=16000, mono=True)
    spec = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)

    hf = spec[freqs > 7000].mean()
    total = spec.mean()

    return float(hf / (total + 1e-6))


In [ ]:
def assess_audio_risk(audio_path):
    feat = extract_features(audio_path)
    feat = scaler.transform([feat])

    ml_prob = model.predict_proba(feat)[0][1]
    replay_prob = min(replay_score(audio_path), 1.0)
    hidden_prob = hidden_command_score(audio_path)

    final_risk = (
        0.6 * ml_prob +
        0.2 * replay_prob +
        0.2 * hidden_prob
    )

    decision = (
        "BLOCK" if final_risk > 0.75 else
        "FLAG" if final_risk > 0.45 else
        "ALLOW"
    )

    return {
        "risk_score": round(final_risk, 3),
        "decision": decision
    }


In [ ]:
result = assess_audio_risk("data/audio/prompt_injection/synthetic_1.mp3")
print(result)


{'risk_score': np.float64(0.61), 'decision': 'FLAG'}


In [ ]:
!pip install -q torch torchaudio librosa numpy matplotlib scikit-learn


In [ ]:
!pip install -q numpy<2.0 librosa soundfile scipy scikit-learn torch openai-whisper tqdm


/bin/bash: line 1: 2.0: No such file or directory


In [ ]:
import os
import librosa
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Config
SAMPLE_RATE = 16000
DURATION = 5
N_MFCC = 40
BATCH_SIZE = 16
EPOCHS = 30        # ↑ increase
LR = 1e-3



In [ ]:
def load_audio(path):
    audio, sr = librosa.load(path, sr=SAMPLE_RATE, mono=True)
    audio = librosa.util.fix_length(audio, size=SAMPLE_RATE * DURATION)
    return audio


In [ ]:

def extract_features(audio):
    mfcc = librosa.feature.mfcc(
        y=audio,
        sr=SAMPLE_RATE,
        n_mfcc=N_MFCC
    )
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)

    features = np.concatenate([
        np.mean(mfcc, axis=1),
        np.mean(delta, axis=1),
        np.mean(delta2, axis=1)
    ])

    return features.astype(np.float32)  # shape = 120



In [ ]:
# Example structure:
# data/
#   safe/
#   scam/

DATA_DIR = "/content/"

AUDIO_EXTS = (".wav", ".mp3", ".m4a")

audio_files = []
labels = []

for label, folder in enumerate(["original", "prompt_injection"]):
    folder_path = os.path.join(DATA_DIR, folder)
    for file in os.listdir(folder_path):
        if file.lower().endswith(AUDIO_EXTS):
            audio_files.append(os.path.join(folder_path, file))
            labels.append(label)

print("Total audio files:", len(audio_files))


Total audio files: 192


In [ ]:
from sklearn.model_selection import train_test_split

X_train_files, X_val_files, y_train, y_val = train_test_split(
    audio_files,
    labels,
    test_size=0.2,
    random_state=42,
    stratify=labels   # VERY IMPORTANT
)


In [ ]:
def build_feature_matrix(files):
    feats = []
    for path in files:
        audio = load_audio(path)
        feats.append(extract_features(audio))
    return np.array(feats)


In [ ]:
X_train = build_feature_matrix(X_train_files)
X_val   = build_feature_matrix(X_val_files)


/tmp/ipython-input-1496612483.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(path, sr=SAMPLE_RATE, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-1496612483.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(path, sr=SAMPLE_RATE, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipython-input-1496612483.py:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(path, sr=SAMPLE_RATE, mono=Tru

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)


In [ ]:
class AudioDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [ ]:

class AudioClassifier(nn.Module):
    def __init__(self, input_dim=120, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
train_ds = AudioDataset(X_train, y_train)
val_ds   = AudioDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE)


Total audio files: 200


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
from collections import Counter

counts = Counter(y_train)
total = sum(counts.values())

weights = [
    total / counts[0],
    total / counts[1]
]

class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)


In [ ]:
# =========================
# Device
# =========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# =========================
# Hyperparameters
# =========================
LR = 3e-4          # stable for small dataset
EPOCHS = 20

# =========================
# Model
# =========================
model = AudioClassifier().to(device)

# =========================
# Loss (with class weights)
# =========================
criterion = nn.CrossEntropyLoss(weight=class_weights)

# =========================
# Optimizer
# =========================
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,
    weight_decay=0.005
)

# =========================
# Training Loop
# =========================
for epoch in range(EPOCHS):
    # ---- Train ----
    model.train()
    train_loss = 0.0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]"):
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)

            outputs = model(x)
            loss = criterion(outputs, y)
            val_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    val_loss /= len(val_loader)
    val_acc = correct / total

    # ---- Logs ----
    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )


Using device: cpu


Epoch 1/20 [Train]: 100%|██████████| 10/10 [00:00<00:00, 179.21it/s]


Epoch 1/20 | Train Loss: 0.7124 | Val Loss: 0.6999 | Val Acc: 0.5641


Epoch 2/20 [Train]: 100%|██████████| 10/10 [00:00<00:00, 195.97it/s]


Epoch 2/20 | Train Loss: 0.6695 | Val Loss: 0.6899 | Val Acc: 0.5897


...

Epoch 18/20 | Train Loss: 0.3923 | Val Loss: 0.6278 | Val Acc: 0.6154


Epoch 19/20 [Train]: 100%|██████████| 10/10 [00:00<00:00, 176.55it/s]


Epoch 19/20 | Train Loss: 0.3306 | Val Loss: 0.6319 | Val Acc: 0.6154


Epoch 20/20 [Train]: 100%|██████████| 10/10 [00:00<00:00, 178.18it/s]

Epoch 20/20 | Train Loss: 0.3335 | Val Loss: 0.6389 | Val Acc: 0.6410


In [ ]:

model.eval()
correct, total = 0, 0

with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        preds = torch.argmax(model(x), dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print("Validation Accuracy:", correct / total)


Validation Accuracy: 0.6410256410256411


In [ ]:
import os
import librosa
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Config
SAMPLE_RATE = 16000
DURATION = 5
N_MFCC = 40
BATCH_SIZE = 16
EPOCHS = 30        # ↑ increase
LR = 1e-3



In [ ]:
import librosa
import numpy as np
import torch


In [ ]:
SAMPLE_RATE = 16000
DURATION = 5
N_MFCC = 40


In [ ]:
def load_audio(path):
    audio, _ = librosa.load(path, sr=SAMPLE_RATE, mono=True)
    audio = librosa.util.fix_length(
        audio,
        size=SAMPLE_RATE * DURATION
    )
    return audio


In [ ]:
def extract_features(audio):
    mfcc = librosa.feature.mfcc(
        y=audio,
        sr=SAMPLE_RATE,
        n_mfcc=N_MFCC
    )
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)

    features = np.concatenate([
        np.mean(mfcc, axis=1),
        np.mean(delta, axis=1),
        np.mean(delta2, axis=1)
    ])

    return features.astype(np.float32)


In [ ]:


def test_audio(audio_path, model, scaler, device):
    model.eval()

    try:
        audio = load_audio(audio_path)
        features = extract_features(audio)
        features = scaler.transform([features])[0]

        x = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            probs = torch.softmax(model(x), dim=1)[0]
            orig_prob = probs[0].item()
            inj_prob  = probs[1].item()

        # 🔥 ORIGINAL-favoring thresholds
        PI_THRESH   = 0.75
        ORIG_THRESH = 0.40

        if inj_prob >= PI_THRESH:
            final_label = "PROMPT_INJECTION"
            confidence = inj_prob
        elif orig_prob > ORIG_THRESH:
            final_label = "ORIGINAL"
            confidence = orig_prob
        else:
            # soft bias toward ORIGINAL
            final_label = "ORIGINAL"
            confidence = orig_prob


        print("=" * 55)
        print("Audio file  :", audio_path)
        print("Prediction  :", final_label)
        print(f"Confidence  : {confidence * 100:.2f}%")
        print("=" * 55)

        return final_label, confidence

    except Exception as e:
        print("❌ Error testing audio:", audio_path)
        print("Reason:", str(e))
        return None, None


In [ ]:
def test_audio_smooth(audio_path, model, scaler, device, runs=5):
    probs = []

    for _ in range(runs):
        audio = load_audio(audio_path)
        audio = audio + 0.01 * np.random.randn(len(audio))  # tiny noise

        features = extract_features(audio)
        features = scaler.transform([features])[0]

        x = torch.tensor(
            features,
            dtype=torch.float32
        ).unsqueeze(0).to(device)

        with torch.no_grad():
            # probability of PROMPT_INJECTION
            p = torch.softmax(model(x), dim=1)[0, 1].item()
            probs.append(p)

    avg_prob = float(np.mean(probs))   # <-- this is inj_prob (smoothed)
    orig_prob = 1 - avg_prob

    # 🔥 ORIGINAL-favoring thresholds
    PI_THRESH   = 0.75
    ORIG_THRESH = 0.50

    if avg_prob >= PI_THRESH:
        final_label = "PROMPT_INJECTION"
        confidence = avg_prob
    elif orig_prob > ORIG_THRESH:
        final_label = "ORIGINAL"
        confidence = orig_prob
    else:
        # soft bias toward ORIGINAL
        final_label = "ORIGINAL"
        confidence = orig_prob

    print("=" * 55)
    print("Audio file :", audio_path)
    print("Avg Prob (PI):", f"{avg_prob:.2f}")
    print("Prediction :", final_label)
    print(f"Confidence : {confidence * 100:.2f}%")
    print("=" * 55)

    return final_label, confidence


In [ ]:
test_audio_smooth(
    "/content/prompt_injection/synthetic (45).wav",
    model,
    scaler,
    device
)


Audio file : /content/prompt_injection/synthetic (45).wav
Avg Prob (PI): 0.84
Prediction : PROMPT_INJECTION
Confidence : 84.26%


('PROMPT_INJECTION', 0.8426075339317322)

In [ ]:
# =========================
# TRAINING FINISHED
# =========================

print("✅ Training completed")

# =========================
# SAVE MODEL FOR FUTURE USE
# =========================

import os
import torch
import joblib

SAVE_DIR = "final_saved_model"
os.makedirs(SAVE_DIR, exist_ok=True)

# 1️⃣ Save model weights
torch.save(model.state_dict(), f"{SAVE_DIR}/audio_classifier.pt")

# 2️⃣ Save scaler (VERY IMPORTANT)
joblib.dump(scaler, f"{SAVE_DIR}/scaler.pkl")

# 3️⃣ Save config
config = {
    "input_dim": 120,   # MFCC + delta + delta2
    "num_classes": 2
}
torch.save(config, f"{SAVE_DIR}/config.pt")

print("✅ Model, scaler, and config saved successfully")
print(f"📁 Files saved inside: {SAVE_DIR}/")


✅ Training completed
✅ Model, scaler, and config saved successfully
📁 Files saved inside: final_saved_model/


In [ ]:
# Quick sanity check
model.load_state_dict(torch.load("saved_model/audio_classifier.pt", map_location=device))
model.eval()

print("✅ Reload test passed")


✅ Reload test passed


In [ ]:
import os

TEST_DIR = "/content/test_audios"
AUDIO_EXTS = (".wav", ".mp3", ".m4a")

for file in os.listdir(TEST_DIR):
    if file.lower().endswith(AUDIO_EXTS):
        test_audio(
            os.path.join(TEST_DIR, file),
            model,
            scaler,
            device
        )


In [ ]:
!pip install -U openai-whisper
!apt-get update -y
!apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.9 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=285f25cc158a7ba9ed2a41ecd0758c197ba58d76a9911c0f221c4ae7c89b13a7
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
import whisper

whisper_model = whisper.load_model("small")

def transcribe_audio(path):
    result = whisper_model.transcribe(path)
    return result["text"]


100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 110MiB/s]


In [ ]:
test_audio = audio_files[0]

audio = load_audio(test_audio)
features = extract_mfcc(audio)

model.eval()
with torch.no_grad():
    logits = model(torch.tensor(features).unsqueeze(0).to(device))
    prediction = torch.argmax(logits, dim=1).item()

print("Prediction:", "SCAM" if prediction == 1 else "SAFE")
print("Transcript:", transcribe_audio(test_audio))


Prediction: SAFE


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcript:  let's say your rectangle which has a curve instead of coming to a point.
